In [1]:
import pandas as pd
import os
import numpy as np
import datetime as dt

from os import listdir
from os.path import isfile, join
from os.path import getmtime

from zebra import remov_duplicates, human_format
from directories import *

In [2]:
idx = pd.IndexSlice

# Sources

## Parametriche

In [3]:
par = {}
for file in parametriche_files:
    file_name=file.split(".")[0]
    par[f"{file_name}"] = pd.read_excel(parametriche+"\\"+file)

In [4]:
cod_prod = pd.DataFrame(par["Classificazione_codice_prodotto"]).loc[:,['MDM', 'GTM BDG', 'Solution','Sub solution',"Codice PRODOTTO"]].assign(**{"Codice PRODOTTO": lambda x: x["Codice PRODOTTO"].astype(str)})

sales_director = pd.DataFrame(par["Sales_director"]).loc[:,["Sales Director", "Area territoriale", "Rete"]].assign(**{"Sales Director": lambda x: x["Sales Director"].str.lower()})

## Parco clienti da Licenze e abbonamenti

In [47]:
dfs = []
for file in abb_files:
    data = pd.read_excel(abbonamenti+"\\"+file, usecols=["Cliente MERCE"], dtype={"Cliente MERCE":str}, na_values="#")
    data["Tipo Licenza"] = file.split("_")[1].split(".")[0]
    data["Ultimo Agg."] = dt.date.fromtimestamp(getmtime(abbonamenti+"\\"+file))
    dfs.append(data)

cli = pd.concat(dfs, axis=0, ignore_index=True)

## Parco clienti da Licenze e abbonamenti

In [6]:
cli_merce = pd.read_excel(anagrafiche+"\\"+"Lista_clienti_merce_da_abbonamenti.xlsb", dtype={"Cliente MERCE":str})

## Abbonamenti disdetti

In [10]:
dis = pd.read_excel(base_dati+r"\Abbonamenti disdetti\Abb_Disdetti_Diretta-Indiretta.xlsb", dtype={"Cliente MERCE":str, "Cliente FATTURA":str, "Codice ABBONAMENTO":str, "Data inizio contratt":object, "Data fine contratto":object,"Data Inizio Abb.":object}, na_values="#")

for col in dis.columns:
    if "Data" in col:
        
        dis[col] = dis[col].apply(lambda x: (float(x) - 25569) * 86400)
        dis[col] = pd.to_datetime(dis[col], unit="s")
    else:
        pass

In [15]:
dis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 449539 entries, 0 to 449538
Data columns (total 13 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   Cliente MERCE              449539 non-null  object        
 1   Cliente FATTURA            449539 non-null  int64         
 2   Sistema                    449539 non-null  object        
 3   Numero SISTEMA             436593 non-null  object        
 4   Stato Sintetico Contratto  449539 non-null  object        
 5   Data inizio contratt       436588 non-null  datetime64[ns]
 6   Data fine contratto        436545 non-null  datetime64[ns]
 7   Codice ABBONAMENTO         449539 non-null  object        
 8   Tipo Abbonamento           449539 non-null  object        
 9   Data Inizio Abb.           449539 non-null  datetime64[ns]
 10  Stato Esteso               449539 non-null  object        
 11  Sales Director             449539 non-null  object  

## Creazione dataframe

In [48]:
def df_extend(df_):

  df = dis.loc[~dis["Cliente MERCE"].isin(cli["Cliente MERCE"]),:].copy()
  df["Sales Director"] = df["Sales Director"].str.lower()
  df = df.merge(cod_prod, how="left", left_on="Codice ABBONAMENTO", right_on="Codice PRODOTTO")\
    .merge(sales_director, how="left", on="Sales Director")
    
  return df

dis_tot = dis.pipe(df_extend)

In [49]:
dis_tot_arca = dis_tot.groupby(["Solution", "Rete", "Area territoriale", "Cliente MERCE"]).agg({"Data fine contratto":max, "Valore PV New":sum}).loc["Arca Evolution"].reset_index()

In [53]:
dis_tot_arca.rename(columns={"Data fine contratto": "Ultima data fine contratto", "Valore PV New":"Valore PV"}, inplace=True)

In [34]:
dis_tot_arca.to_excel(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Analisi clienti\Clienti per area territoriale\Clienti diretta\EED e LA\Data export\Lista clienti disdetti.xlsx", index=False)